# Install block2

Clone github repo:

In [1]:
import os
from getpass import getpass
user = 'hczhai'
pw = getpass('Password: ')
os.system('git clone https://{}:{}@github.com/block-hczhai/block2.git'.format(user, pw))
del pw

Password: ··········


Build `block2` python interface:

In [2]:
!pip3 install -q pybind11 mkl mkl-include
%cd block2
!mkdir build
!cd build && MKLROOT=/usr/local cmake .. -DUSE_MKL=ON -DBUILD_LIB=ON && time make

     |████████████████████████████████| 296kB 2.9MB/s 
     |████████████████████████████████| 880kB 9.2MB/s 
/content/block2
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/gcc
-- Check for working C compiler: /usr/bin/gcc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/g++
-- Check for working CXX compiler: /usr/bin/g++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found PythonInterp: /usr/bin/python3 (found version "3.6.9") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.6m.so (found suitable version "3.6.9", minimum required is "3.6") 
-- PROJECT_NAME = block2
-- PYTHON_VERSION_MAJOR = 3
-- PYTHON_VERSION_MINOR = 6
-- PYTH

# Hubbard Model

In [3]:
from pyblock2.algebra import *
import numpy as np
import itertools

Perfom DMRG inside C++ block2 to obtain a ground-state MPS:

In [4]:
%%time
with HamilTools.hubbard(n_sites=8, u=2, t=1) as hamil:
    mps = hamil.get_ground_state_mps(bond_dim=250)
    mpo = hamil.get_mpo()

Ground State Energy =  -6.2256341283857175
CPU times: user 612 ms, sys: 33.7 ms, total: 646 ms
Wall time: 828 ms


Compress NC MPO (by removing zero two-electron operators for Hubbard model):

In [5]:
print('MPO (NC) =         ', mpo.show_bond_dims())
mpo.compress(cutoff=1E-12)
print('MPO (compressed) = ', mpo.show_bond_dims())

MPO (NC) =          46|82|142|226|334|466|622
MPO (compressed) =  6|6|6|6|6|6|6


Check ground-state energy:

In [6]:
print('MPS energy = ', mps @ mpo @ mps)

MPS energy =  -6.225634128385716


Check that ground-state MPS is normalized:

In [7]:
print('MPS = ', mps.show_bond_dims())
print('MPS norm = ', mps.norm())

MPS =  4|16|64|235|64|16|4
MPS norm =  1.0000000000000004


### MPS Scaling

MPS scaling (by scaling the first MPS tensor):

In [8]:
print('2 MPS = ', (2 * mps).show_bond_dims())
print((2 * mps).norm())

2 MPS =  4|16|64|235|64|16|4
2.000000000000001


Check the first MPS tensor:

In [9]:
mps[0]

  0 (Q=) (< N=0 SZ=0 PG=0 >, < N=0 SZ=0 PG=0 >) (R=) array([[0.37390503]])
  1 (Q=) (< N=1 SZ=1/2 PG=0 >, < N=1 SZ=1/2 PG=0 >) (R=) array([[-0.60017023]])
  2 (Q=) (< N=1 SZ=-1/2 PG=0 >, < N=1 SZ=-1/2 PG=0 >) (R=) array([[0.60016213]])
  3 (Q=) (< N=2 SZ=0 PG=0 >, < N=2 SZ=0 PG=0 >) (R=) array([[0.37389323]])

In [10]:
(2 * mps)[0]

  0 (Q=) (< N=0 SZ=0 PG=0 >, < N=0 SZ=0 PG=0 >) (R=) array([[0.74781006]])
  1 (Q=) (< N=1 SZ=1/2 PG=0 >, < N=1 SZ=1/2 PG=0 >) (R=) array([[-1.20034045]])
  2 (Q=) (< N=1 SZ=-1/2 PG=0 >, < N=1 SZ=-1/2 PG=0 >) (R=) array([[1.20032426]])
  3 (Q=) (< N=2 SZ=0 PG=0 >, < N=2 SZ=0 PG=0 >) (R=) array([[0.74778645]])

### MPS Addition

MPS addition will increase the bond dimension:

In [11]:
mps_add = mps + mps
print('MPS + MPS = ', mps_add.show_bond_dims())
print(mps_add.norm())

MPS + MPS =  8|32|128|470|128|32|8
2.000000000000001


Check the overlap `<2MPS|MPS+MPS>`:

In [12]:
%%time
mps_add @ (2 * mps)

CPU times: user 22 ms, sys: 0 ns, total: 22 ms
Wall time: 23.1 ms


4.000000000000004

### MPS Canonicalization

Left canonicalization:

In [13]:
lmps = mps_add.deep_copy()
lmps.canonicalize(center=mps_add.n_sites - 1)
print('L-MPS = ', lmps.show_bond_dims())

L-MPS =  4|16|64|255|128|32|8


Right canonicalization:

In [14]:
rmps = mps_add.deep_copy()
rmps.canonicalize(center=0)
print('R-MPS = ', rmps.show_bond_dims())

R-MPS =  8|32|128|255|64|16|4


Check the overlap `<LMPS|RMPS>`:

In [15]:
%%time
lmps @ rmps

CPU times: user 23.2 ms, sys: 0 ns, total: 23.2 ms
Wall time: 24.7 ms


4.000000000000005

### MPS Compression

Compression will first do canonicalization from left to right, then do SVD from right to left.

This can further decrease bond dimension of MPS.

In [16]:
print('MPS + MPS = ', mps_add.show_bond_dims())
mps_add.compress(cutoff=1E-12)
print('MPS + MPS = ', mps_add.show_bond_dims())
print(mps_add.norm())

MPS + MPS =  8|32|128|470|128|32|8
MPS + MPS =  4|16|64|235|64|16|4
2.0000000000000027


### MPS Subtraction

Subtractoin will also increase bond dimension:

In [17]:
mps_minus = mps - mps
print('MPS - MPS = ', mps_minus.show_bond_dims())

MPS - MPS =  8|32|128|470|128|32|8


After compression, this is zero:

In [18]:
mps_minus.compress(cutoff=1E-12)
print('MPS - MPS = ', mps_minus.show_bond_dims())
print(mps_minus.norm())

MPS - MPS =  0|0|0|0|0|0|0
0.0


### MPS Bond Dimension Truncation

Apply MPO two times to MPS:

In [19]:
%%time
hhmps = mpo @ (mpo @ mps)
print(hhmps.show_bond_dims())
print(np.sqrt(hhmps @ mps))

144|576|2304|8460|2304|576|144
6.2256341345763895
CPU times: user 676 ms, sys: 76.6 ms, total: 753 ms
Wall time: 692 ms


MPS compression can be used to reduce bond dimension (to FCI):

In [20]:
%%time
cps_error = hhmps.compress(cutoff=1E-12)
print('error = ', cps_error)
print(hhmps.show_bond_dims())
print(np.sqrt(hhmps @ mps))

error =  0.0
4|16|64|256|64|16|4
6.225634134576391
CPU times: user 209 ms, sys: 141 ms, total: 350 ms
Wall time: 184 ms


Truncation to bond dimension 100 will introduce a small error:

In [21]:
cps_error = hhmps.compress(k=100, cutoff=1E-12)
print('error = ', cps_error)
print(hhmps.show_bond_dims())
print(np.sqrt(hhmps @ mps))

error =  0.00135950632741534
4|16|64|100|64|16|4
6.225634131631153


Truncation to bond dimension 30 will introduce a larger error:

In [22]:
cps_error = hhmps.compress(k=30, cutoff=1E-12)
print('error = ', cps_error)
print(hhmps.show_bond_dims())
print(np.sqrt(hhmps @ mps))

error =  0.12822806032019754
4|16|30|30|30|16|4
6.22553507208518


### MPO-MPO Contraction

One can also first contract two MPO:

In [23]:
%%time
h2 = mpo @ mpo
print(h2.show_bond_dims())

36|36|36|36|36|36|36
CPU times: user 216 ms, sys: 6.05 ms, total: 222 ms
Wall time: 223 ms


Check expectation value:

In [24]:
%%time
print(np.sqrt((h2 @ mps) @ mps))

6.22563413457639
CPU times: user 730 ms, sys: 52.6 ms, total: 783 ms
Wall time: 725 ms


### MPO Bond Dimension Truncation

Compression MPO (keeping accuracy):

In [25]:
cps_error = h2.compress(cutoff=1E-12)
print('error = ', cps_error)
print(h2.show_bond_dims())
print(np.sqrt((h2 @ mps) @ mps))

error =  1.725092949971858e-13
16|17|17|17|17|17|16
6.2256341345763895


MPO Truncated to bond dimension 15:

In [26]:
cps_error = h2.compress(k=15, cutoff=1E-12)
print('error = ', cps_error)
print(h2.show_bond_dims())
print(np.sqrt((h2 @ mps) @ mps))

error =  173.35865814081157
15|15|15|15|15|15|15
6.097436276051796


MPO Truncated to bond dimension 12:

In [27]:
cps_error = h2.compress(k=12, cutoff=1E-12)
print('error = ', cps_error)
print(h2.show_bond_dims())
print(np.sqrt((h2 @ mps) @ mps))

error =  221.69663116602277
12|12|12|12|12|12|12
5.763720368829762


# N2 (10o, 7e) (STO3G)

### Initialization

Get ground-state MPS (from FCIDUMP):

In [28]:
%%time
with HamilTools.from_fcidump('data/N2.STO3G.FCIDUMP') as hamil:
    mps = hamil.get_ground_state_mps(bond_dim=250)
    mpo = hamil.get_mpo()
    orb_sym = hamil.hamil.orb_sym

Ground State Energy =  -107.65412244667549
CPU times: user 1.54 s, sys: 101 ms, total: 1.64 s
Wall time: 1.57 s


Check MPO:

In [29]:
print('MPO = ', mpo.show_bond_dims())
mpo.compress(cutoff=1E-12)
print('MPO = ', mpo.show_bond_dims())

MPO =  54|90|150|234|342|474|630|810|1014
MPO =  16|42|80|90|80|62|67|30|12


Check MPS:

In [30]:
print('MPS = ', mps.show_bond_dims())
print(mps.norm())

MPS =  4|16|50|78|62|43|30|8|4
1.0


Check ground-state energy:

In [31]:
%%time
mps @ mpo @ mps

CPU times: user 835 ms, sys: 110 ms, total: 945 ms
Wall time: 875 ms


-107.6541224466753

### MPO-MPS Contraction

In [32]:
%%time
hmps = mpo @ mps
print(hmps.show_bond_dims())
print(hmps.norm())

68|688|4050|7098|5022|2709|2040|248|52
107.65412244669304
CPU times: user 1.58 s, sys: 233 ms, total: 1.82 s
Wall time: 1.28 s


The result MPS can be compressed:

In [33]:
hmps.compress(cutoff=1E-12)
print(hmps.show_bond_dims())
print(hmps.norm())

4|16|58|106|96|50|30|8|4
107.65412244669302


MPO truncation to bond dimension 50:

In [34]:
cmpo = mpo.deep_copy()
cps_error = cmpo.compress(k=50, cutoff=1E-12)
print('error = ', cps_error)
print(cmpo.show_bond_dims())

error =  234.6641660949853
16|42|50|50|50|50|50|30|12


Apply contracted MPO to MPS:

In [35]:
hmps = cmpo @ mps
print(hmps.show_bond_dims())
print(hmps.norm())

68|688|2550|3978|3162|2193|1530|248|52
107.60510153352527


### Determinants

MPS can be created from occupation numbers:

In [36]:
det1 = HamilTools.get_determinant_mps([
    1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
    1, 1, 1, 1, 1, 1, 1, 1, 0, 0], orb_sym)
print('DET1 = ', det1.show_bond_dims())

DET1 =  1|1|1|1|1|1|1|1|1


Check the overlap:

In [37]:
print(det1 @ mps)

-0.002059485244140836


If the determinant belongs to another symmetry sector, the overlap should be zero:

In [38]:
det2 = HamilTools.get_determinant_mps([
    1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
    1, 1, 1, 1, 1, 1, 1, 1, 0, 0], orb_sym)
print('DET2 = ', det2.show_bond_dims())
print(det2 @ mps)

DET2 =  1|1|1|1|1|1|1|1|1
0.0


Check the overlap for all doubly occupied determinants:

In [39]:
%%time
coeffs = []
for ocp in itertools.combinations(range(10), 7):
    det = [0] * mps.n_sites * 2
    for t in ocp:
        det[t * 2] = det[t * 2 + 1] = 1
    det_mps = HamilTools.get_determinant_mps(det, orb_sym)
    coeffs.append(det_mps @ mps)
    print(np.array(det).reshape((-1, 2)).sum(axis=1), "%10.5f" % coeffs[-1])

[2 2 2 2 2 2 2 0 0 0]    0.01154
[2 2 2 2 2 2 0 2 0 0]   -0.00186
[2 2 2 2 2 2 0 0 2 0]    0.01156
[2 2 2 2 2 2 0 0 0 2]   -0.00186
[2 2 2 2 2 0 2 2 0 0]    0.01270
[2 2 2 2 2 0 2 0 2 0]   -0.95751
[2 2 2 2 2 0 2 0 0 2]    0.13129
[2 2 2 2 2 0 0 2 2 0]    0.13129
[2 2 2 2 2 0 0 2 0 2]   -0.03846
[2 2 2 2 2 0 0 0 2 2]    0.01270
[2 2 2 2 0 2 2 2 0 0]   -0.00086
[2 2 2 2 0 2 2 0 2 0]    0.01314
[2 2 2 2 0 2 2 0 0 2]   -0.00241
[2 2 2 2 0 2 0 2 2 0]   -0.00241
[2 2 2 2 0 2 0 2 0 2]    0.00056
[2 2 2 2 0 2 0 0 2 2]   -0.00086
[2 2 2 2 0 0 2 2 2 0]    0.05471
[2 2 2 2 0 0 2 2 0 2]   -0.00684
[2 2 2 2 0 0 2 0 2 2]    0.05471
[2 2 2 2 0 0 0 2 2 2]   -0.00684
[2 2 2 0 2 2 2 2 0 0]   -0.00002
[2 2 2 0 2 2 2 0 2 0]    0.00099
[2 2 2 0 2 2 2 0 0 2]   -0.00014
[2 2 2 0 2 2 0 2 2 0]   -0.00014
[2 2 2 0 2 2 0 2 0 2]    0.00004
[2 2 2 0 2 2 0 0 2 2]   -0.00002
[2 2 2 0 2 0 2 2 2 0]    0.00032
[2 2 2 0 2 0 2 2 0 2]   -0.00005
[2 2 2 0 2 0 2 0 2 2]    0.00032
[2 2 2 0 2 0 0 2 2 2]   -0.00005
[2 2 2 0 0

Check the sum of probabilities:

In [40]:
print((np.array(coeffs) ** 2).sum())

0.9626219907526998


# Finite-Temperature DMRG

### Time-Step Targeting Method (Reference)

Time-step targeting imaginary TE implemented in C++ block2:

In [41]:
def time_step_targeting(hamil, bond_dim=200, beta_step=0.01, mu=-1.0, pattern=False):
    from block2 import VectorUInt16, QCTypes, TETypes, SeqTypes, TruncPatternTypes
    from block2.sz import (
        MPOQC, SimplifiedMPO, AncillaMPO,
        RuleQC, MovingEnvironment, ImaginaryTE
    )
    hamil.hamil.mu = mu
    mpo = MPOQC(hamil.hamil, QCTypes.Conventional)
    mpo = SimplifiedMPO(AncillaMPO(mpo), RuleQC())
    hamil.hamil.opf.seq.mode = SeqTypes.Simple
    mpo.const_e = 0.0
    with hamil.get_thermal_limit_mps_block2(dot=2) as mps:
        me = MovingEnvironment(mpo, mps, mps, "TE")
        me.init_environments(True)
        te = ImaginaryTE(me, VectorUInt16([bond_dim]), TETypes.RK4, 6)
        if pattern:
            te.trunc_pattern = TruncPatternTypes.TruncAfterOdd
        te.solve(1, beta_step / 2, mps.center == 0)
        ener = te.energies[-1]
    mpo.deallocate()
    return ener

Perform time-step targeting FT-DMRG for Hydrogen chain (STO6G):

(Error is the difference from exact diagonalization result. )

$\Delta\beta = 0.01, M = 200$:

In [42]:
%%time
with HamilTools.from_fcidump('data/H8.STO6G.R1.8.FCIDUMP') as hamil:
    ener = time_step_targeting(hamil, bond_dim=200, beta_step=0.01)
    print('Energy = ', ener)
    print('Error  = ', ener - 0.2408363230374028)

Energy =  0.24098644954367984
Error  =  0.0001501265062770485
CPU times: user 4.24 s, sys: 155 ms, total: 4.39 s
Wall time: 4.4 s


$\Delta\beta = 0.01, M = 500$:

In [43]:
%%time
with HamilTools.from_fcidump('data/H8.STO6G.R1.8.FCIDUMP') as hamil:
    ener = time_step_targeting(hamil, bond_dim=500, beta_step=0.01)
    print('Energy = ', ener)
    print('Error  = ', ener - 0.2408363230374028)

Energy =  0.24083635000433087
Error  =  2.696692807946377e-08
CPU times: user 8.37 s, sys: 256 ms, total: 8.63 s
Wall time: 8.95 s


$\Delta\beta = 0.02, M = 200$:

In [44]:
%%time
with HamilTools.from_fcidump('data/H8.STO6G.R1.8.FCIDUMP') as hamil:
    ener = time_step_targeting(hamil, bond_dim=200, beta_step=0.02)
    print('Energy = ', ener)
    print('Error  = ', ener - 0.1711364110139819)

Energy =  0.17142705521198692
Error  =  0.0002906441980050356
CPU times: user 6.34 s, sys: 190 ms, total: 6.53 s
Wall time: 6.72 s


$\Delta\beta = 0.02, M = 500$:

In [45]:
%%time
with HamilTools.from_fcidump('data/H8.STO6G.R1.8.FCIDUMP') as hamil:
    ener = time_step_targeting(hamil, bond_dim=500, beta_step=0.02)
    print('Energy = ', ener)
    print('Error  = ', ener - 0.1711364110139819)

Energy =  0.17113657684061942
Error  =  1.6582663753261784e-07
CPU times: user 23.1 s, sys: 472 ms, total: 23.6 s
Wall time: 24.3 s


### Ancilla Approach

Fourth-order Runge-Kutta:

In [46]:
# apply exp(-beta / 2 * mpo) * mps
def rk4_apply(mpo, mps, beta, bond_dim=200):
    k1 = (-beta / 2) * (mpo @ mps)
    k1.compress(k=bond_dim, cutoff=1E-12)
    k2 = (-beta / 2) * (mpo @ (0.5 * k1 + mps))
    k2.compress(k=bond_dim, cutoff=1E-12)
    k3 = (-beta / 2) * (mpo @ (0.5 * k2 + mps))
    k3.compress(k=bond_dim, cutoff=1E-12)
    k4 = (-beta / 2) * (mpo @ (k3 + mps))
    k4.compress(k=bond_dim, cutoff=1E-12)
    mps = mps + (1 / 6) * k1 + (1 / 3) * k2 + (1 / 3) * k3 + (1 / 6) * k4
    mps.compress(k=bond_dim, cutoff=1E-12)
    return mps

Load 8-site Hydrogen chain model:

In [47]:
with HamilTools.from_fcidump('data/H8.STO6G.R1.8.FCIDUMP') as hamil:
    mps = hamil.get_thermal_limit_mps()
    mpo = hamil.get_mpo(mu=-1.0, ancilla=True)
    mpo.const_e = 0.0

Check MPS and MPO (number of sites = 16):

In [48]:
print('MPS = ', mps.show_bond_dims())
print('MPO = ', mpo.show_bond_dims())
mpo.compress(cutoff=1E-8)
print('MPO = ', mpo.show_bond_dims())

MPS =  4|1|4|1|4|1|4|1|4|1|4|1|4|1|4
MPO =  46|46|82|82|142|142|226|226|334|334|466|466|622|622|1
MPO =  16|16|46|46|84|84|122|122|84|84|46|46|16|16|1


Initial MPS energy:

In [49]:
init_e = (mps | (mpo @ mps)) / (mps | mps)
print('Initial Energy = ', init_e)
print('Error          = ', init_e - 0.3124038410492045)

Initial Energy =  0.3124038410492058
Error          =  1.3322676295501878e-15


Imaginary-time evolution:

$\Delta\beta = 0.01, M = 200$:

In [50]:
%%time
fmps = rk4_apply(mpo, mps, beta=0.01, bond_dim=200)
ener = (fmps | (mpo @ fmps)) / (fmps | fmps)
print('Energy = ', ener)
print('Error  = ', ener - 0.2408363230374028)

Energy =  0.24098203293759857
Error  =  0.0001457099001957729
CPU times: user 5min 51s, sys: 34.1 s, total: 6min 25s
Wall time: 3min 23s


$\Delta\beta = 0.02, M = 200$:

In [51]:
%%time
fmps = rk4_apply(mpo, mps, beta=0.02, bond_dim=200)
ener = (fmps | (mpo @ fmps)) / (fmps | fmps)
print('Energy = ', ener)
print('Error  = ', ener - 0.1711364110139819)

Energy =  0.1714263005809038
Error  =  0.0002898895669219137
CPU times: user 5min 51s, sys: 33.8 s, total: 6min 25s
Wall time: 3min 21s


### Matrix Product Density Operator (MPDO) Approach

Load 8-site Hydrogen chain model:

In [52]:
with HamilTools.from_fcidump('data/H8.STO6G.R1.8.FCIDUMP') as hamil:
    mps = hamil.get_thermal_limit_mps()
    mpo = hamil.get_mpo(mu=-1.0, ancilla=False)
    mpo.const_e = 0.0

Translate initial ancilla MPS to MPDO by contracting physical and ancilla sites:

In [53]:
tensors = []
for i in range(mps.n_sites // 2):
    tensors.append(Tensor.contract(mps[i * 2], mps[i * 2 + 1], [-1], [0]))
mps = MPO(tensors=tensors)

Check MPS and MPO (number of sites = 8):

In [54]:
print('MPS = ', mps.show_bond_dims())
print('MPO = ', mpo.show_bond_dims())
mpo.compress(cutoff=1E-8)
print('MPO = ', mpo.show_bond_dims())

MPS =  1|1|1|1|1|1|1
MPO =  46|82|142|226|334|466|622
MPO =  16|46|84|122|84|46|16


Initial MPS energy:

In [55]:
init_e = (mps | (mpo @ mps)) / (mps | mps)
print('Initial Energy = ', init_e)
print('Error          = ', init_e - 0.3124038410492045)

Initial Energy =  0.3124038410492051
Error          =  6.106226635438361e-16


Imaginary-time evolution:

$\Delta\beta = 0.01, M = 200$:

In [56]:
%%time
fmps = rk4_apply(mpo, mps, beta=0.01, bond_dim=200)
ener = (fmps | (mpo @ fmps)) / (fmps | fmps)
print('Energy = ', ener)
print('Error  = ', ener - 0.2408363230374028)

Energy =  0.24083635154179525
Error  =  2.8504392457495697e-08
CPU times: user 10min 31s, sys: 49.2 s, total: 11min 20s
Wall time: 6min 2s


$\Delta\beta = 0.02, M = 200$:

In [57]:
%%time
fmps = rk4_apply(mpo, mps, beta=0.02, bond_dim=200)
ener = (fmps | (mpo @ fmps)) / (fmps | fmps)
print('Energy = ', ener)
print('Error  = ', ener - 0.1711364110139819)

Energy =  0.17113665985616144
Error  =  2.488421795565632e-07
CPU times: user 10min 29s, sys: 44.8 s, total: 11min 13s
Wall time: 5min 57s


### Time-Step Targeting Method (Alternating Truncation)

$\Delta\beta = 0.01, M = 200$:

In [58]:
%%time
with HamilTools.from_fcidump('data/H8.STO6G.R1.8.FCIDUMP') as hamil:
    ener = time_step_targeting(hamil, bond_dim=200, beta_step=0.01, pattern=True)
    print('Energy = ', ener)
    print('Error  = ', ener - 0.2408363230374028)

Energy =  0.240836350724998
Error  =  2.7687595216541183e-08
CPU times: user 32.6 s, sys: 615 ms, total: 33.2 s
Wall time: 33.1 s


$\Delta\beta = 0.02, M = 200$:

In [59]:
%%time
with HamilTools.from_fcidump('data/H8.STO6G.R1.8.FCIDUMP') as hamil:
    ener = time_step_targeting(hamil, bond_dim=200, beta_step=0.02, pattern=True)
    print('Energy = ', ener)
    print('Error  = ', ener - 0.1711364110139819)

Energy =  0.1711366190114786
Error  =  2.079974967261755e-07
CPU times: user 1min 14s, sys: 1.01 s, total: 1min 15s
Wall time: 1min 15s
